In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
!pip install sentence_transformers==2.7.0

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/Colab_Notebooks/DSCI691/MedNLI


/content/gdrive/MyDrive/Colab_Notebooks/DSCI691/MedNLI


In [ ]:
import math
from sentence_transformers import models, losses, datasets, SentenceTransformer, InputExample
import logging
from datetime import datetime
import pandas as pd
import torch

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Define model parameters
model_name = "distilbert-base-uncased"
train_batch_size = 4
max_seq_length = 128
num_epochs = 10

In [ ]:
# Load the pre-trained model fine-tuned on SNLI and MultiNLI
pretrained_model_path = "/content/gdrive/MyDrive/Colab_Notebooks/output/exp1+ BERT+SNLIdistilbert-base-uncased-2024-06-09_15-41-18"
model = SentenceTransformer(pretrained_model_path)

In [ ]:
# Load the MedNLI dataset
def load_mednli(file_path):
    return pd.read_json(file_path, lines=True)

train_df = load_mednli('mli_train_v1.jsonl')
dev_df = load_mednli('mli_dev_v1.jsonl')
test_df = load_mednli('mli_test_v1.jsonl')

In [ ]:
# Reduce the dataset size for testing (optional)
train_df = train_df.sample(500)  # Sample 500 examples from the training set
dev_df = dev_df.sample(100)  # Sample 100 examples from the validation set
test_df = test_df.sample(100)  # Sample 100 examples from the test set

In [ ]:
# Preprocess the data
train_df = train_df.rename(columns={"sentence1": "sent1", "sentence2": "sent2", "gold_label": "label"})
dev_df = dev_df.rename(columns={"sentence1": "sent1", "sentence2": "sent2", "gold_label": "label"})
test_df = test_df.rename(columns={"sentence1": "sent1", "sentence2": "sent2", "gold_label": "label"})

In [ ]:
# Filter out invalid labels
train_df = train_df[train_df['label'].isin(['entailment', 'neutral', 'contradiction'])]
dev_df = dev_df[dev_df['label'].isin(['entailment', 'neutral', 'contradiction'])]
test_df = test_df[test_df['label'].isin(['entailment', 'neutral', 'contradiction'])]

In [ ]:
# Map labels to integers
label_mapping = {'entailment': 0, 'neutral': 1, 'contradiction': 2}
train_df['labels'] = train_df['label'].map(label_mapping)
dev_df['labels'] = dev_df['label'].map(label_mapping)
test_df['labels'] = test_df['label'].map(label_mapping)

In [ ]:
# Convert DataFrame to InputExample
def convert_to_input_examples(df):
    return [InputExample(texts=[row['sent1'], row['sent2']], label=row['labels']) for idx, row in df.iterrows()]

train_samples = convert_to_input_examples(train_df)
dev_samples = convert_to_input_examples(dev_df)
test_samples = convert_to_input_examples(test_df)

In [ ]:
# Create DataLoader
train_dataloader = datasets.NoDuplicatesDataLoader(train_samples, batch_size=train_batch_size)
dev_dataloader = datasets.NoDuplicatesDataLoader(dev_samples, batch_size=train_batch_size)
test_dataloader = datasets.NoDuplicatesDataLoader(test_samples, batch_size=train_batch_size)

In [ ]:
# Define the loss function
train_loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
# Save path of the model
model_save_path = (
    "/content/gdrive/MyDrive/Colab_Notebooks/output/exp2+BERT+SNLI+MULTNLI+MEDNLI" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)

In [ ]:
# Configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
logger.info("Warmup-steps: {}".format(warmup_steps))

In [ ]:
# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
    use_amp=False,  # Disable mixed precision training for CPU
)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]